## SupraMolSim Analysis workflows focus on exploting its core methods (showcased in general workflows) within parameter sweeps to explore the imaging space!


### In order to do so, we will use the Experiment module, desinged to ease the navigation of SupraMolSim functionallity at any point in the workflow.
The Experiment itself allows you to initialise a demo experiment


In [ ]:
from supramolsim.experiments import create_experiment_parametrisation
from supramolsim.analysis.scripts import parameter_sweep_reps, analyse_sweep

In [ ]:
# generated at user interface or stored programatically
selected_mods = dict(
    STED_demo=None,
    Confocal_demo=None,
)
structure_and_labels = dict(
    structure_id = "7R5K",
    structure_label = "7R5K_Nup96_Cterminal_direct",
    fluorophore_id = "AF647"
)
defects_eps_d = dict(
    eps1=300,
    eps2=600
)
savging = dict(
    experiment_id = "SupraMolSim_experiment",
    output_directory = "",
    save=False
)

sweep_pars = dict(
    labelling_efficiency=dict(
        start=0.3,
        end=1,
        nintervals=3, 
        ideal=1
    ),
    defects=dict(
        start=0,
        end=0.5,
        nintervals=3,
        ideal=0
    )
)
total_combinations = sweep_pars["defects"]["nintervals"] * sweep_pars["labelling_efficiency"]["nintervals"]

Experiment_generator = create_experiment_parametrisation(
    structure_and_labels=structure_and_labels,
    modalities_acquisition=selected_mods,
    savging=savging,
    defects_params=defects_eps_d,
    use_locals=True
)

In [ ]:
replicas = 3
sweep_out, sweep_out_pars, ref_out = parameter_sweep_reps(
    Experiment=Experiment_generator,
    sweep_parameters=sweep_pars,
    repetitions=replicas)

In [ ]:
sweep_analyse_pars = dict(
    STED_demo=dict(
        metric="ssim",
        subregion=[20,44]
    ),
    Confocal_demo=dict(
        metric="ssim",
        subregion=False
    )
)
data_frame, qries, references = analyse_sweep(
    sweep_out,
    sweep_out_pars,
    ref_out,
    sweep_analyse_pars)

In [ ]:
from supramolsim.analysis._plots import show_references, show_example
from supramolsim.analysis._plots import pivot_dataframes, sns_heatmap_pivots

In [ ]:
param_names = list(data_frame.columns[0:2])
print(param_names)
dataframe_pivots = pivot_dataframes(data_frame, param_names)
conditions_cmaps = dict(
    Confocal_demo="mako",
    STED_demo="rocket"
)
sns_heatmap_pivots(df_pivots=dataframe_pivots, conditions_cmaps=conditions_cmaps, annotations=False)